<a href="https://colab.research.google.com/github/parinnay/quant_finance/blob/main/momentum_investment_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats

In [ ]:
!pip install xlsxwriter

     |████████████████████████████████| 149 kB 8.7 MB/s 


In [ ]:
import xlsxwriter

In [ ]:
stocks = pd.read_csv("sp_500_stocks.csv")

In [ ]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [ ]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()

In [ ]:
data

{'avg10Volume': 90774498,
 'avg30Volume': 99009055,
 'beta': 1.2670278300401772,
 'companyName': 'Apple Inc',
 'day200MovingAvg': 165.71,
 'day30ChangePercent': 0.07790637121300689,
 'day50MovingAvg': 170.22,
 'day5ChangePercent': 0.04505716243254212,
 'dividendYield': 0.005075497767764153,
 'employees': 149461,
 'exDividendDate': '2022-01-29',
 'float': 0,
 'marketcap': 3020527504309,
 'maxChangePercent': 69.96514494736223,
 'month1ChangePercent': 0.07957056609597074,
 'month3ChangePercent': -0.001199156186260345,
 'month6ChangePercent': 0.25065836832890404,
 'nextDividendDate': '',
 'nextEarningsDate': '2022-04-23',
 'peRatio': 29.32990651470329,
 'sharesOutstanding': 16785779516,
 'ttmDividendRate': 0.880802266626103,
 'ttmEPS': 6.11,
 'week52change': 0.4659061045935102,
 'week52high': 186.28,
 'week52highSplitAdjustOnly': 187.48,
 'week52low': 125.36,
 'week52lowSplitAdjustOnly': 127.05,
 'year1ChangePercent': 0.5042821828150902,
 'year2ChangePercent': 1.8341853691715087,
 'year5Ch

In [ ]:
data["year1ChangePercent"]

0.5042821828150902

In [ ]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))

my_columns = ["Ticker", "Price", "One-Year Price Return", "Number of Shares to Buy"]

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(","):
        try:
            final_dataframe = final_dataframe.append(
                pd.Series(
                    [symbol,
                    data[symbol]["price"],
                    data[symbol]["stats"]["year1ChangePercent"],
                    "N/A"],
                    index = my_columns), 
                    ignore_index = True
                )
        except:
            pass

In [ ]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,138.68,0.091916,N/A
1,AAL,18.42,-0.258369,N/A
2,AAP,209.67,0.163737,N/A
3,AAPL,182.64,0.492665,N/A
4,ABBV,163.95,0.613888,N/A
...,...,...,...,...
497,YUM,119.94,0.120563,N/A
498,ZBH,129.30,-0.206588,N/A
499,ZBRA,433.70,-0.099779,N/A
500,ZION,68.72,0.243892,N/A


In [ ]:
final_dataframe.sort_values("One-Year Price Return", ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)

In [ ]:
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,274,LB,83.66,2.309715,N/A
1,148,DVN,61.86,1.919756,N/A
2,316,MRO,25.37,1.420805,N/A
3,42,APA,41.64,1.385595,N/A
4,89,CF,106.66,1.309692,N/A
5,344,NVDA,276.83,1.180016,N/A
6,356,OXY,59.33,1.160746,N/A
7,313,MOS,68.70,1.118124,N/A
8,298,MCHP,75.78,1.085074,N/A
9,109,COP,104.00,0.980375,N/A


In [ ]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio:")
    try:
        float(portfolio_size)
    except:
        print("This is not a number")
        portfolio_size = input("Enter the size of your portfolio:")

In [ ]:
portfolio_input()

Enter the size of your portfolio:80


In [ ]:
position_size = float(portfolio_size)/len(final_dataframe.index)
position_size

1.6

In [ ]:
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i, "Price"])

In [ ]:
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,274,LB,83.66,2.309715,0
1,148,DVN,61.86,1.919756,0
2,316,MRO,25.37,1.420805,0
3,42,APA,41.64,1.385595,0
4,89,CF,106.66,1.309692,0
5,344,NVDA,276.83,1.180016,0
6,356,OXY,59.33,1.160746,0
7,313,MOS,68.70,1.118124,0
8,298,MCHP,75.78,1.085074,0
9,109,COP,104.00,0.980375,0


In [ ]:
#improved strategy

In [ ]:
hqm_columns = ['Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
               
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

In [ ]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            hqm_dataframe = hqm_dataframe.append(
                                            pd.Series([symbol, 
                                                    data[symbol]['quote']['latestPrice'],
                                                    'N/A',
                                                    data[symbol]['stats']['year1ChangePercent'],
                                                    'N/A',
                                                    data[symbol]['stats']['month6ChangePercent'],
                                                    'N/A',
                                                    data[symbol]['stats']['month3ChangePercent'],
                                                    'N/A',
                                                    data[symbol]['stats']['month1ChangePercent'],
                                                    'N/A',
                                                    'N/A'
                                                    ], 
                                                    index = hqm_columns), 
                                            ignore_index = True)
        except:
            pass

In [ ]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,135.90,N/A,0.09195,N/A,-0.157165,N/A,-0.162373,N/A,0.013329,N/A,N/A
1,AAL,18.27,N/A,-0.252986,N/A,-0.145312,N/A,-0.001144,N/A,0.035347,N/A,N/A
2,AAP,214.21,N/A,0.163832,N/A,-0.01211,N/A,-0.102561,N/A,0.037925,N/A,N/A
3,AAPL,182.57,N/A,0.515234,N/A,0.257904,N/A,-0.001149,N/A,0.078621,N/A,N/A
4,ABBV,166.18,N/A,0.630787,N/A,0.55775,N/A,0.220456,N/A,0.097506,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,120.35,N/A,0.119489,N/A,-0.009611,N/A,-0.125081,N/A,-0.024206,N/A,N/A
498,ZBH,132.70,N/A,-0.200637,N/A,-0.142013,N/A,0.005359,N/A,0.01817,N/A,N/A
499,ZBRA,420.30,N/A,-0.095658,N/A,-0.18623,N/A,-0.283771,N/A,0.027737,N/A,N/A
500,ZION,66.02,N/A,0.238633,N/A,0.068749,N/A,0.074487,N/A,-0.052266,N/A,N/A


In [ ]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
        for time_period in time_periods:
            hqm_dataframe[f"{time_period} Price Return"] = pd.to_numeric(hqm_dataframe[f"{time_period} Price Return"])
            hqm_dataframe.loc[row, f"{time_period} Return Percentile"] = stats.percentileofscore(hqm_dataframe[f"{time_period} Price Return"].to_list(), 
                                                                                                 hqm_dataframe.loc[row, f"{time_period} Price Return"])

In [ ]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,135.90,N/A,0.091950,42.430279,-0.157165,8.964143,-0.162373,15.936255,0.013329,40.239044,N/A
1,AAL,18.27,N/A,-0.252986,4.581673,-0.145312,11.553785,-0.001144,52.988048,0.035347,50.996016,N/A
2,AAP,214.21,N/A,0.163832,55.976096,-0.012110,35.856574,-0.102561,29.083665,0.037925,52.390438,N/A
3,AAPL,182.57,N/A,0.515234,90.239044,0.257904,85.059761,-0.001149,52.788845,0.078621,68.7251,N/A
4,ABBV,166.18,N/A,0.630787,94.621514,0.557750,96.613546,0.220456,90.438247,0.097506,76.89243,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,120.35,N/A,0.119489,48.406375,-0.009611,37.051793,-0.125081,22.908367,-0.024206,22.50996,N/A
498,ZBH,132.70,N/A,-0.200637,7.768924,-0.142013,11.952191,0.005359,56.175299,0.018170,43.625498,N/A
499,ZBRA,420.30,N/A,-0.095658,17.131474,-0.186230,6.772908,-0.283771,1.195219,0.027737,47.609562,N/A
500,ZION,66.02,N/A,0.238633,68.924303,0.068749,54.38247,0.074487,73.505976,-0.052266,13.944223,N/A


In [ ]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [ ]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,135.90,N/A,0.091950,42.430279,-0.157165,8.964143,-0.162373,15.936255,0.013329,40.239044,26.89243
1,AAL,18.27,N/A,-0.252986,4.581673,-0.145312,11.553785,-0.001144,52.988048,0.035347,50.996016,30.02988
2,AAP,214.21,N/A,0.163832,55.976096,-0.012110,35.856574,-0.102561,29.083665,0.037925,52.390438,43.326693
3,AAPL,182.57,N/A,0.515234,90.239044,0.257904,85.059761,-0.001149,52.788845,0.078621,68.7251,74.203187
4,ABBV,166.18,N/A,0.630787,94.621514,0.557750,96.613546,0.220456,90.438247,0.097506,76.89243,89.641434
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,120.35,N/A,0.119489,48.406375,-0.009611,37.051793,-0.125081,22.908367,-0.024206,22.50996,32.719124
498,ZBH,132.70,N/A,-0.200637,7.768924,-0.142013,11.952191,0.005359,56.175299,0.018170,43.625498,29.880478
499,ZBRA,420.30,N/A,-0.095658,17.131474,-0.186230,6.772908,-0.283771,1.195219,0.027737,47.609562,18.177291
500,ZION,66.02,N/A,0.238633,68.924303,0.068749,54.38247,0.074487,73.505976,-0.052266,13.944223,52.689243


In [ ]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

In [ ]:
portfolio_input()

Enter the size of your portfolio:10000


In [ ]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe.sort_values(by = "HQM Score", ascending = False, inplace = True)
hqm_dataframe.reset_index(inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
hqm_dataframe

,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,42,APA,43.18,4,1.370732,98.605578,0.995619,99.203187,0.551622,98.207171,0.233832,97.808765,98.456175
1,11,ADM,91.36,2,0.593667,94.023904,0.500534,95.01992,0.345887,95.418327,0.133623,90.438247,93.7251
2,4,ABBV,166.18,1,0.630787,94.621514,0.557750,96.613546,0.220456,90.438247,0.097506,76.89243,89.641434
3,39,ANTM,513.15,0,0.411737,86.055777,0.300670,88.247012,0.079873,75.298805,0.114732,85.059761,83.665339
4,47,ATO,120.38,1,0.244191,69.920319,0.377657,91.832669,0.165625,86.653386,0.107341,81.2749,82.420319
5,5,ABC,159.35,1,0.324958,77.091633,0.279837,86.454183,0.166151,86.85259,0.095627,76.095618,81.623506
6,37,ANET,141.03,1,0.888358,96.613546,0.626664,97.410359,-0.035031,43.027888,0.123609,88.645418,81.424303
7,40,AON,333.21,0,0.433939,88.047809,0.125834,65.537849,0.091413,77.290837,0.117491,86.055777,79.233068
8,15,AEP,101.88,1,0.220286,65.737052,0.237142,82.868526,0.132234,82.669323,0.113956,84.661355,78.984064
9,26,ALL,141.11,1,0.241310,69.521912,0.102557,59.163347,0.201436,90.039841,0.132259,90.239044,77.241036


In [ ]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [ ]:
writer.save()